In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
#selenium libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException, TimeoutException
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import ui

import time as t
import pandas as pd
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


In [3]:
import requests
import urllib.parse
import os
import shutil
pd.set_option("max_rows", None)

In [4]:
def get_drive_launch(url):
    """
    This function initiates the Chrome browser's driver with the url passed as a parameter and returns the driver instance.
    Parameters:
        url - url of the website
    return:
        driver - driver instance of the chrome browser    
    """
    #creating driver instance
    driver = webdriver.Chrome('./driver/chromedriver.exe')
    driver.maximize_window()
    
    #defining implicit wait
    driver.implicitly_wait(10)
    
    #launching the url
    driver.get(url)
    
    return driver

In [5]:
def scrape_cardata_olx():
    """
    This method will scrape cars data from the OLX site.
    """
    url = 'https://www.olx.in/cars_c84'
                            
    olx_brand = []
    olx_model = []
    olx_variant = []
    olx_manu_year = []
    olx_kilometers = []
    olx_fuel_type = []
    olx_owners = []
    olx_location = []
    olx_gear_type = []
    olx_price = []
    
    d = get_drive_launch(url)
    locations = d.find_elements_by_xpath("//div[@data-aut-id='collapsible_topLocations']//li//li")
    
    for i in range(len(locations)):
        try:
            locations[i].click()
        except StaleElementReferenceException:
            d.find_element_by_xpath("//a[@data-aut-id='location_India']//*[contains(text(),'India')]").click()
            t.sleep(3)
            if i < 4:
                locations = d.find_elements_by_xpath("//div[@data-aut-id='collapsible_topLocations']//li//li")
            else:
                locations = d.find_elements_by_xpath("//div[@data-aut-id='collapsible_topLocations']//li//ul/div/li")
            locations[i].click()
        except IndexError:
            break
        #print(d.find_element_by_xpath("//p[@id='adsResultsIn']/b").text)
        try:
            ol_location = d.find_element_by_xpath("//p[@id='adsResultsIn']/b").text
        except NoSuchElementException:
            continue
        gear_shifts = ['Automatic','Manual']            
        brands = d.find_elements_by_xpath("//*[contains(text(),'All Brands')]/following-sibling::div/div/div")
        
        for j in range(len(brands)):
            try:
                brands[j].click()
            except IndexError:
                break
            t.sleep(2)
            
            for gear in gear_shifts:
                try:
                    gear_ele = ui.WebDriverWait(d, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'_2yv8x')]//*[contains(text(),'"+gear+"')]")))
                    gear_ele.click()
                except TimeoutException:
                    element = d.find_element_by_xpath("//div[contains(@class,'_2yv8x')]//*[contains(text(),'"+gear+"')]")
                    actions = ActionChains(d)
                    actions.move_to_element(element).perform()
                    #d.execute_script("window.scrollTo(0,document.body.scrollHeight)")
                    gear_ele = ui.WebDriverWait(d, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'_2yv8x')]//*[contains(text(),'"+gear+"')]")))
                    gear_ele.click()
                    #d.find_element_by_xpath("//div[contains(@class,'_2yv8x')]//*[contains(text(),'"+gear+"')]").click()
                    
                t.sleep(2)
                cars = d.find_elements_by_xpath("//li[@data-aut-id='itemBox']")
                olx_gear_type.append(gear)
                
                for k in range(len(cars)):
                    
                    try:
                        cars[k].click()
                        t.sleep(2)    
                        try:
                            d.find_element_by_xpath("//*[contains(text(),'Oops')]")
                            d.get(url)
                            t.sleep(2)
                            cars = d.find_elements_by_xpath("//li[@data-aut-id='itemBox']")
                            continue
                        except NoSuchElementException:
                            pass
                        try:
                            brand = ui.WebDriverWait(d, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@data-aut-id='breadcrumb']//li[7]")))
                            olx_brand.append(brand.text.split(' in ')[0].strip())
                            #olx_brand.append(d.find_element_by_xpath("//div[@data-aut-id='breadcrumb']//li[7]").text.split(' in ')[0].strip())
                        except NoSuchElementException:
                            t.sleep(2)
                            olx_brand.append(d.find_element_by_xpath("//span[@data-aut-id='value_make']").text)
                        #print(d.find_element_by_xpath("//div[@data-aut-id='breadcrumb']//li[7]").text.split(' in ')[0].strip())
                        except TimeoutException:
                            t.sleep(2)
                            try:
                                olx_brand.append(d.find_element_by_xpath("//div[@data-aut-id='breadcrumb']//li[4]").text)
                            except NoSuchElementException:
                                t.sleep(2)
                                olx_brand.append(d.find_element_by_xpath("//div[@data-aut-id='breadcrumb']//li[7]").text.split(' in ')[0].strip())
                            except NoSuchElementException:
                                t.sleep(2)
                                olx_brand.append(d.find_element_by_xpath("//span[@data-aut-id='value_make']").text)
                        
                        try:
                            model = ui.WebDriverWait(d, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@data-aut-id='breadcrumb']//li[8]")))
                            olx_model.append(model.text.split(' in ')[0].strip())
                        except TimeoutException:
                            t.sleep(2)
                            olx_model.append(d.find_element_by_xpath("//div[@data-aut-id='breadcrumb']//li[4]").text.split(' in ')[0].strip())
                        
                        try:
                            olx_variant.append(d.find_element_by_xpath("//span[@data-aut-id='value_variant']").text)
                            olx_location.append(ol_location)
                        except NoSuchElementException:
                            try:
                                data = d.find_element_by_xpath("//div[@class='_1lysw']").text.split(' ',1)
                                olx_variant.append(data[0].split('-')[0])
                                olx_location.append(ol_location)
                            except NoSuchElementException:
                                olx_variant.append('--')
                                olx_location.append(ol_location)

                        try:
                            olx_manu_year.append(d.find_element_by_xpath("//span[@data-aut-id='value_year']").text)
                        except NoSuchElementException:
                            try:
                                data = d.find_element_by_xpath("//div[@class='_1lysw']").text.split(' ',1)
                                try:
                                    olx_manu_year.append(data[1])
                                except IndexError:
                                    olx_manu_year.append(d.find_element_by_xpath("//div[@data-aut-id='itemTitle']").text.split('(')[1].strip(')'))
                            except NoSuchElementException:
                                olx_manu_year.append('--')
                                
                        try:
                            olx_kilometers.append(d.find_element_by_xpath("//span[@data-aut-id='value_mileage']").text)
                        except NoSuchElementException:
                            try:
                                olx_kilometers.append(d.find_element_by_xpath("//div[contains(@class, '_1Rnja') and (contains(text(),'Km'))]").text)
                            except NoSuchElementException:
                                olx_kilometers.append('--')
                                
                        try:
                            olx_fuel_type.append(d.find_element_by_xpath("//span[@data-aut-id='value_petrol']").text)
                        except NoSuchElementException:
                            try:
                                olx_fuel_type.append(d.find_element_by_xpath("//div[contains(@class, '_1Rnja')]").text)
                            except NoSuchElementException:
                                olx_fuel_type.append('--')
                                
                        try:
                            olx_owners.append(d.find_element_by_xpath("//span[@data-aut-id='value_first_owner']").text)
                        except NoSuchElementException:
                            try:
                                olx_owners.append(d.find_element_by_xpath("//div[contains(@class, '_1z3tZ')]").text)
                            except NoSuchElementException:
                                olx_owners.append('--')
                        
                        try:
                            olx_price.append(d.find_element_by_xpath("//span[@data-aut-id='itemPrice']").text)
                        except NoSuchElementException:
                            try:
                                olx_price.append(d.find_element_by_xpath("//div[contains(@class, '_2r-aw')]").text)
                            except NoSuchElementException:
                                olx_price.append('--')
                                          
                        d.back()
                        t.sleep(2)
                        cars = d.find_elements_by_xpath("//li[@data-aut-id='itemBox']")
                    except ElementClickInterceptedException:
                        break            
            
            try:
                #exit_brand = ui.WebDriverWait(d, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='_19Q27']/div/div[not(contains(text(),'Automatic')) and not(contains(text(),'Manual'))]")))
                exit_brand = ui.WebDriverWait(d, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='_2JV-T']//*[contains(text(),'Clear All')]")))
                exit_brand.click()
            except ElementClickInterceptedException:
                try:
                    exit_brand = ui.WebDriverWait(d, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='_19Q27']/div/div[not(contains(text(),'Automatic')) and not(contains(text(),'Manual'))]")))
                    exit_brand.click()
                except TimeoutException:
                    d.back()
                    t.sleep(2)
                    try:
                        exit_brand = d.find_element_by_xpath("//a[@class='_19Q27']/div/div[not(contains(text(),'Automatic')) and not(contains(text(),'Manual'))]")
                        exit_brand.click()
                    except (ElementClickInterceptedException, NoSuchElementException):
                        break
                except ElementClickInterceptedException:
                    d.get(url)
            except:
                break
                    
            t.sleep(2)
            brands = d.find_elements_by_xpath("//*[contains(text(),'All Brands')]/following-sibling::div/div/div")
        try:
            d.find_element_by_xpath("//*[contains(text(),'temporarily')]")
            d.get(url)
        except NoSuchElementException:
            pass
        t.sleep(2)
        try:
            d.find_element_by_xpath("//div[@data-aut-id='collapsible_topLocations']//*[contains(text(),'India')]").click()
        except (NoSuchElementException,ElementClickInterceptedException):
            d.get(url)
            
        t.sleep(2)
        if i < 4:
            locations = d.find_elements_by_xpath("//div[@data-aut-id='collapsible_topLocations']//li//li")
        else:
            locations = d.find_elements_by_xpath("//div[@data-aut-id='collapsible_topLocations']//li//ul/div/li")
    
    return (olx_brand,olx_model,olx_variant,olx_manu_year,olx_kilometers,olx_fuel_type,olx_owners,olx_location,olx_price)
    #return data


In [9]:
(olx_brand,olx_model,olx_variant,olx_manu_year,olx_kilometers,olx_fuel_type,olx_owners,olx_location,olx_price) = scrape_cardata_olx()
#data_frame = scrape_cardata_olx()


In [10]:
for i in (olx_brand,olx_model,olx_variant,olx_manu_year,olx_kilometers,olx_fuel_type,olx_owners,olx_location,olx_price):
    print(len(i))

1164
1164
1164
1164
1164
1164
1164
1164
1164


In [11]:
data_frame = pd.DataFrame({'brand':olx_brand, 'model':olx_model, 'variant':olx_variant,
                        'manufactured_year':olx_manu_year, 'kilometers_driven':olx_kilometers,
                        'fuel_type':olx_fuel_type, 'owners':olx_owners, 'location':olx_location,
                        'price':olx_price})

In [12]:
data_frame.to_csv("olx_data.csv")

In [41]:
temp_url = "https://www.cardekho.com/used-cars+in+ahmedabad"
#https://www.cardekho.com/used-maruti+cars+in+ahmedabad
temp_url = temp_url.split('-')
temp_url.insert(1,"-maruthi+")
print("".join(temp_url))

https://www.cardekho.com/used-maruthi+cars+in+ahmedabad


In [50]:
len("mahindra renault".split())

2

In [63]:
def scrape_cardata_cardekho():
    """
    This method will scrape cars data from the OLX site.
    """
    url = 'https://www.cardekho.com/usedCars'
                            
    cd_brand = []
    cd_model = []
    cd_variant = []
    cd_manu_year = []
    cd_kilometers = []
    cd_fuel_type = []
    cd_owners = []
    cd_location = []
    cd_price = []
    
    brand_name = ''
    model = ''
    variant = ''
    manu_year = ''
    kilometers = ''
    fuel_type = ''
    owners = ''
    location = ''
    price = ''
    
    
    
    d = get_drive_launch(url)
    
    locations = d.find_elements_by_xpath("//section[@data-track-component='Used cars in India by city']//li")
    lo = d.find_elements_by_xpath("//section[@data-track-component='Used cars in India by city']//li//span[3]")
    for i in range(len(locations)):
        try:
            location = lo[i].text.strip()
            locations[i].click()
            t.sleep(2)
            print("cars in ",location)
            temp_url = d.current_url
            #brands = d.find_elements_by_xpath("//ul[@data-track-section='Search by brand or model']//li[@class='accordion brandli makeList']/label")
            brand_names = d.find_elements_by_xpath("//ul[@data-track-section='Search by brand or model']//li[@class='accordion brandli makeList']/label")
            
            for j in range(len(brand_names)):
                try:
                    brand_name = brand_names[j].get_attribute('title')
                    if len(brand_name.split()) > 1:
                        brand = brand_name.split()
                        brand.insert(1,"-")
                        brand_name = "".join(brand)
                    tempurl = temp_url.split('-')
                    tempurl.insert(1,"-"+brand_name+"+")
                    tempurl = "".join(tempurl)
                    
                    d.get(tempurl)
                                        
                    t.sleep(2)
                    #print(d.find_element_by_xpath("//h1").text)
                                        
                    cars = d.find_elements_by_xpath("//div[@class='gsc_col-md-4 gsc_col-sm-6 gsc_col-xs-12']//div[@class='imgarea hover']")
                    car_names = d.find_elements_by_xpath("//div[@class='gsc_col-md-4 gsc_col-sm-6 gsc_col-xs-12']//div[@class='holder']//div[@class='gsc_col-xs-7 carsName']/a")
                    car_variants = d.find_elements_by_xpath("//div[@class='gsc_col-md-4 gsc_col-sm-6 gsc_col-xs-12']//div[@class='holder']//div[@class='gsc_col-xs-7 carsName']/div[1]")
                    
                except Exception as e:
                    print("erroring out", e)
                    
                k = 0
                if len(cars) > 10:
                    cars = cars[:10]
                for k in range(len(cars)):
                    try:
                        model = car_names[k].text.strip()
                        model = model.split(brand_name)
                        try:
                            manu_year = model[0].strip()
                        except:
                            manu_year = '--'
                        try:
                            model = model[1].strip()
                        except:
                            model = '--'
                        try:
                            variant = car_variants[k].text.strip()
                        except:
                            variant = '--'
                        cars[k].click()
                    except StaleElementReferenceException:
                        #d.get(d.current_url)
                        t.sleep(2)
                        cars = d.find_elements_by_xpath("//div[@class='gsc_col-md-4 gsc_col-sm-6 gsc_col-xs-12']//div[@class='imgarea hover']")
                        car_names = d.find_elements_by_xpath("//div[@class='gsc_col-md-4 gsc_col-sm-6 gsc_col-xs-12']//div[@class='holder']//div[@class='gsc_col-xs-7 carsName']/a")
                        car_variants = d.find_elements_by_xpath("//div[@class='gsc_col-md-4 gsc_col-sm-6 gsc_col-xs-12']//div[@class='holder']//div[@class='gsc_col-xs-7 carsName']/div[1]")
                        model = car_names[k].text.strip()
                        model = model.split(brand_name)
                        try:
                            manu_year = model[0].strip()
                        except:
                            manu_year = '--'
                        try:
                            model = model[1].strip()
                        except:
                            model = '--'
                        try:
                            variant = car_variants[k].text.strip()
                        except:
                            variant = '--'
                        cars[k].click()
                    except IndexError:
                        break
                        
                    try:
                        kilometers = d.find_element_by_xpath("//div[contains(text(),'kms')]").text.strip()
                    except NoSuchElementException:
                        try:
                            kilometers = d.find_element_by_xpath("//*[contains(text(),'Kms')]").text.strip()
                        except NoSuchElementException:
                            kilometers = '--'
                    try:                        
                        owners = d.find_element_by_xpath("//*[contains(text(),'Owner')][2]").text.strip()
                    except NoSuchElementException:
                        try:
                            owners = d.find_element_by_xpath("//*[contains(text(),'Owner')]").text.strip()
                        except NoSuchElementException:
                            owners = '--'
                        
                    try:
                        fuel_type = d.find_element_by_xpath("//*[contains(text(),'Owner')]/ancestor::li/following-sibling::li[1]//div[@class='iconDetail']").text.strip()
                    except NoSuchElementException:
                        try:
                            fuel_type = d.find_element_by_xpath("//*[@src='https://stimg.cardekho.com/pwa/img/CDPrime/Fuel.svg']/../div").text.strip()
                        except NoSuchElementException:
                            try:
                                fuel_type = d.find_element_by_xpath("//*[contains(text(),'Fuel')]/following-sibling::div").text.strip()
                            except NoSuchElementException:
                                fuel_type = '--'
                    
                    try:
                        price = d.find_element_by_xpath("//div[@class='priceSection']/span[2]").text.strip()
                    except NoSuchElementException:
                        try:
                            price = d.find_element_by_xpath("//span[@class='amount']").text.strip()
                        except NoSuchElementException:
                            price = '--'
                    
                    cd_brand.append(brand_name)
                    cd_model.append(model)
                    cd_variant.append(variant)
                    cd_manu_year.append(manu_year)
                    cd_kilometers.append(kilometers)
                    cd_fuel_type.append(fuel_type)
                    cd_owners.append(owners)
                    cd_location.append(location)
                    cd_price.append(price)

                    d.back()
                    t.sleep(2)
                    cars = d.find_elements_by_xpath("//div[@class='gsc_col-md-4 gsc_col-sm-6 gsc_col-xs-12']//div[@class='imgarea hover']")
                    car_names = d.find_elements_by_xpath("//div[@class='gsc_col-md-4 gsc_col-sm-6 gsc_col-xs-12']//div[@class='holder']//div[@class='gsc_col-xs-7 carsName']/a")
                    car_variants = d.find_elements_by_xpath("//div[@class='gsc_col-md-4 gsc_col-sm-6 gsc_col-xs-12']//div[@class='holder']//div[@class='gsc_col-xs-7 carsName']/div[1]")
                    if len(cars) > 10:
                        cars = cars[:10]
                
                
                brand_names = d.find_elements_by_xpath("//ul[@data-track-section='Search by brand or model']//li[@class='accordion brandli makeList']/label")
            
            t.sleep(2)
            d.get(url)
            t.sleep(2)
            locations = d.find_elements_by_xpath("//section[@data-track-component='Used cars in India by city']//li")
            lo = d.find_elements_by_xpath("//section[@data-track-component='Used cars in India by city']//li//span[3]")
            i += 1
        except ElementClickInterceptedException:
            d.get(url)
            t.sleep(2)
            locations = d.find_elements_by_xpath("//section[@data-track-component='Used cars in India by city']//li")
            lo = d.find_elements_by_xpath("//section[@data-track-component='Used cars in India by city']//li//span[3]")
            i += 1
    d.quit()
    return (cd_brand,cd_model,cd_variant,cd_manu_year,cd_kilometers,cd_fuel_type,cd_owners,cd_location,cd_price)
    #return data_frame

In [64]:
(cd_brand,cd_model,cd_variant,cd_manu_year,cd_kilometers,cd_fuel_type,cd_owners,cd_location,cd_price) = scrape_cardata_cardekho()

cars in  Ahmedabad
cars in  Bangalore
cars in  Chennai
cars in  Delhi NCR
cars in  Gurgaon
cars in  Hyderabad
cars in  Jaipur
cars in  Kolkata
cars in  Mumbai
cars in  New Delhi
cars in  Noida
cars in  Pune


In [65]:
data_frame = pd.DataFrame({'brand':cd_brand, 'model':cd_model, 'variant':cd_variant,
                        'manufactured_year':cd_manu_year, 'kilometers_driven':cd_kilometers,
                        'fuel_type':cd_fuel_type, 'owners':cd_owners, 'location':cd_location,
                        'price':cd_price})
    
data_frame

,brand,model,variant,manufactured_year,kilometers_driven,fuel_type,owners,location,price
0,Maruti,Eeco,5 Seater AC BSIV,2016,"45,347 Kms",Petrol,1st Owner,Ahmedabad,"3,74,500"
1,Maruti,Eeco,5 Seater AC,2020,"17,116 Kms",Petrol,1st Owner,Ahmedabad,"4,74,500"
2,Maruti,Swift Dzire,VXI 1.2,2019,"11,935 Kms",Petrol,1st Owner,Ahmedabad,"6,91,000"
3,Maruti,Wagon R,VXI,2014,"131,125 Kms",Petrol,1st Owner,Ahmedabad,"3,09,500"
4,Maruti,Wagon R,VXI,2014,"42,381 Kms",Petrol,1st Owner,Ahmedabad,"3,61,000"
5,Maruti,Eeco,5 Seater AC BSIV,2019,"14,161 Kms",Petrol,1st Owner,Ahmedabad,"4,48,500"
6,Maruti,Swift Dzire,VXI 1.2,2019,"9,303 Kms",Petrol,1st Owner,Ahmedabad,"6,88,000"
7,Maruti,Wagon R,VXI BS IV,2018,"23,197 Kms",Petrol,1st Owner,Ahmedabad,"4,41,500"
8,Maruti,Wagon R,VXI Plus,2017,"43,031 Kms",Petrol,1st Owner,Ahmedabad,"4,08,500"
9,Maruti,S-Presso,VXI,2019,"24,230 Kms",Petrol,1st Owner,Ahmedabad,"4,15,500"


In [66]:
data_frame.to_csv('cardekho_data.csv')